<a href="https://colab.research.google.com/github/aneeshoberoi/cbs_ba2/blob/master/Philadelphia_Police_Dept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Is there evidence of racial disparity in the Philadelphia Police Dept. today?
Scope: Traffic stops by PPD between 2014-2017

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
!ls "/content/gdrive/My Drive/BA2/Case data files/Module 1/Open Policing Project"

'File 1.csv'  'File 2.csv'  'File 3.csv'


In [59]:
import pandas as pd
import datetime as dt
import statsmodels.formula.api as sm

In [40]:
df_file1=pd.read_csv("/content/gdrive/My Drive/BA2/Case data files/Module 1/Open Policing Project/File 1.csv", sep=',', parse_dates=[['date', 'time']])

In [43]:
df_file1.dtypes

date_time           datetime64[ns]
location                    object
lat                        float64
lng                        float64
district                     int64
subject_age                float64
subject_race                object
subject_sex                 object
search_conducted             int64
contraband_found           float64
dtype: object

In [44]:
df_file1.set_index(['date_time'], drop=False)

,date_time,location,lat,lng,district,subject_age,subject_race,subject_sex,search_conducted,contraband_found
date_time,,,,,,,,,,
2014-01-01 08:30:00,2014-01-01 08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19,31.0,black,male,0,NaN
2014-01-01 08:53:00,2014-01-01 08:53:00,CHESTNUT ST / S SCHUYLKILL AV W,39.953124,-75.181936,18,38.0,black,male,0,NaN
2014-01-01 09:00:00,2014-01-01 09:00:00,N 52ND ST / GAINOR RD,39.989757,-75.229019,19,19.0,black,male,0,NaN
2014-01-01 09:45:00,2014-01-01 09:45:00,N 56TH ST / LANCASTER AV,39.980411,-75.234049,19,44.0,black,male,0,NaN
2014-01-01 10:10:00,2014-01-01 10:10:00,5600 BLOCK CHRISTIAN ST,39.948407,-75.235530,18,33.0,black,male,0,NaN
...,...,...,...,...,...,...,...,...,...,...
2017-12-31 23:40:00,2017-12-31 23:40:00,300 BLOCK N 63RD ST,39.967380,-75.245838,19,63.0,black,male,0,NaN
2017-12-31 23:41:00,2017-12-31 23:41:00,4100 BLOCK N BROAD ST,40.015354,-75.149985,39,60.0,black,female,0,NaN
2017-12-31 23:45:00,2017-12-31 23:45:00,300 BLOCK E GURNEY ST,39.990239,-75.125950,24,34.0,hispanic,male,0,NaN


In [45]:
df_file1.head(5)

,date_time,location,lat,lng,district,subject_age,subject_race,subject_sex,search_conducted,contraband_found
0,2014-01-01 08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19,31.0,black,male,0,NaN
1,2014-01-01 08:53:00,CHESTNUT ST / S SCHUYLKILL AV W,39.953124,-75.181936,18,38.0,black,male,0,NaN
2,2014-01-01 09:00:00,N 52ND ST / GAINOR RD,39.989757,-75.229019,19,19.0,black,male,0,NaN
3,2014-01-01 09:45:00,N 56TH ST / LANCASTER AV,39.980411,-75.234049,19,44.0,black,male,0,NaN
4,2014-01-01 10:10:00,5600 BLOCK CHRISTIAN ST,39.948407,-75.235530,18,33.0,black,male,0,NaN


In [46]:
df_file1.set_index('date_time')

,location,lat,lng,district,subject_age,subject_race,subject_sex,search_conducted,contraband_found
date_time,,,,,,,,,
2014-01-01 08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19,31.0,black,male,0,NaN
2014-01-01 08:53:00,CHESTNUT ST / S SCHUYLKILL AV W,39.953124,-75.181936,18,38.0,black,male,0,NaN
2014-01-01 09:00:00,N 52ND ST / GAINOR RD,39.989757,-75.229019,19,19.0,black,male,0,NaN
2014-01-01 09:45:00,N 56TH ST / LANCASTER AV,39.980411,-75.234049,19,44.0,black,male,0,NaN
2014-01-01 10:10:00,5600 BLOCK CHRISTIAN ST,39.948407,-75.235530,18,33.0,black,male,0,NaN
...,...,...,...,...,...,...,...,...,...
2017-12-31 23:40:00,300 BLOCK N 63RD ST,39.967380,-75.245838,19,63.0,black,male,0,NaN
2017-12-31 23:41:00,4100 BLOCK N BROAD ST,40.015354,-75.149985,39,60.0,black,female,0,NaN
2017-12-31 23:45:00,300 BLOCK E GURNEY ST,39.990239,-75.125950,24,34.0,hispanic,male,0,NaN


In [48]:
file1=pd.DataFrame(df_file1)

In [28]:
file1.contraband_found.describe()

count    43597.000000
mean         0.276808
std          0.447426
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: contraband_found, dtype: float64

In [31]:
file1.groupby('subject_race').key.count()

subject_race
black       706853
hispanic    112315
white       215314
Name: key, dtype: int64

In [32]:
file1.groupby('subject_race').search_conducted.sum()

subject_race
black       31030
hispanic     4951
white        7616
Name: search_conducted, dtype: int64

In [33]:
file1.groupby('subject_race').contraband_found.sum()

subject_race
black       8411.0
hispanic    1342.0
white       2315.0
Name: contraband_found, dtype: float64

In [ ]:
# Logistic Regression: Search Conducted v Race

# **Pre-processing Section**

In [57]:
#dummies
dummies_district=pd.get_dummies(file1['district'],prefix='district_')
dummies_gender=pd.get_dummies(file1['subject_sex'],prefix='gender_')
dummies_race=pd.get_dummies(file1['subject_race'],prefix='subject_race_')


file1=pd.concat([file1,dummies_district,dummies_gender,dummies_race],axis=1)

In [98]:
# feature engineering#
#file1.date_time.hour.describe()
file1['hour']=file1['date_time'].dt.hour
file1['is_dark']= [1 if x >= 16 or x <=5 else 0 for x in file1['hour']]

In [99]:
file1.groupby('hour').is_dark.max()

hour
0     1
1     1
2     1
3     1
4     1
5     1
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
Name: is_dark, dtype: int64

In [36]:
final_model=sm.logit(formula="search_conducted ~ subject_race", data=file1)

In [37]:
final_model=final_model.fit()

Optimization terminated successfully.
         Current function value: 0.174543
         Iterations 7


In [38]:
print(final_model.summary())

                           Logit Regression Results                           
Dep. Variable:       search_conducted   No. Observations:              1034482
Model:                          Logit   Df Residuals:                  1034479
Method:                           MLE   Df Model:                            2
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:               0.0008903
Time:                        18:13:39   Log-Likelihood:            -1.8056e+05
converged:                       True   LL-Null:                   -1.8072e+05
Covariance Type:            nonrobust   LLR p-value:                 1.328e-70
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -3.0810      0.006   -530.678      0.000      -3.092      -3.070
subject_race[T.hispanic]     0.0043      0.016      0.277      0.782      -0.026       0.

In [39]:
final_model2=sm.logit(formula="contraband_found ~ subject_race", data=file1)
final_model2=final_model2.fit()
print(final_model2.summary())

Optimization terminated successfully.
         Current function value: 0.589529
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:       contraband_found   No. Observations:                43597
Model:                          Logit   Df Residuals:                    43594
Method:                           MLE   Df Model:                            2
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:               0.0006511
Time:                        18:16:30   Log-Likelihood:                -25702.
converged:                       True   LL-Null:                       -25718.
Covariance Type:            nonrobust   LLR p-value:                 5.343e-08
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.9892      0.013    -77.460      0.000      -1.014

In [ ]:
# controlling for district

In [88]:

final_model3=sm.logit(formula="search_conducted ~ subject_race + is_dark", data=file1)
final_model3=final_model3.fit()
print(final_model3.summary())

Optimization terminated successfully.
         Current function value: 0.174511
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       search_conducted   No. Observations:              1034482
Model:                          Logit   Df Residuals:                  1034478
Method:                           MLE   Df Model:                            3
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:                0.001071
Time:                        19:14:45   Log-Likelihood:            -1.8053e+05
converged:                       True   LL-Null:                   -1.8072e+05
Covariance Type:            nonrobust   LLR p-value:                 1.263e-83
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -3.1298      0.008   -370.336      0.000      -3.146

In [100]:
final_model3=sm.logit(formula="contraband_found ~ subject_race + is_dark", data=file1)
final_model3=final_model3.fit()
print(final_model3.summary())

Optimization terminated successfully.
         Current function value: 0.589446
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:       contraband_found   No. Observations:                43597
Model:                          Logit   Df Residuals:                    43593
Method:                           MLE   Df Model:                            3
Date:                Sun, 19 Jul 2020   Pseudo R-squ.:               0.0007930
Time:                        19:18:29   Log-Likelihood:                -25698.
converged:                       True   LL-Null:                       -25718.
Covariance Type:            nonrobust   LLR p-value:                 7.253e-09
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.0458      0.025    -42.472      0.000      -1.094